In [2]:
import pandas as pd
import numpy as np
import vnstock as vn
import calculation as cal

In [3]:
data = cal.DataProcessor.load_data('VCB') 
# data = data.set_index('time')
#Tính trung bình 20 phiên gần nhất
mean_20 = data['volume'].rolling(window=20).mean() 
#Tính on-balance volume
data['on-balance_volume'] = data['volume']/mean_20 

data['OBV_label'] = data['on-balance_volume'].apply(cal.DataProcessor.label_values)
#Giá đóng cửa ngày t-1
data['close_t_minus_1'] = data['close'].shift(1) 
data['bar_type'] = data.apply(cal.DataProcessor.compare_close_prices, axis=1)   
data['para'] =abs(data['close'] - data['open'])/(data['high'] - data['low'])

data['label_spread'] = data['para'].apply(cal.DataProcessor.label_spread)
data['close_bar_label'] = data.apply(cal.DataProcessor.label_close_bar, axis=1)


In [4]:
data.head()

,time,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread,close_bar_label
0,2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium,bottom-third
1,2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low,middle-third
2,2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium,top-third
3,2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high,bottom-third
4,2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high,top-third


In [5]:

# df_filtered = data[data['bar_type'] == 'up-bar']
# df_filtered


# Pattern of Weakness

## Weakness A
- down bar
- high volume
- medium or low spread
- close: middle-third or bottom-third

In [6]:
# def is_weakness_a_signal(data):
#     if (data['bar_type'] == 'down-bar') and \
#         (data['OBV_label']=='high') and\
#         ((data['label_spread'] == 'low') or (data['label_spread']=='medium')) and \
#         ((data['close_bar_label'] == 'middle-third') or (data['close_bar_label']=='bottom-third')):
#         return True
#     else:
#         return False

In [7]:
def is_weakness_a_signal(data):
    if (data['bar_type'] == 'down-bar') and (data['close_bar_label'] == 'middle-third'):
        return True
    else:
        return False

In [8]:
# Sample data
sample_data = {
    'bar_type': 'down-bar',
    'OBV_label': 'high',
    'label_spread': 'low',
    'close_bar_label': 'middle-third'
}
result = is_weakness_a_signal(sample_data)
print("Is weakness a signal?", result)

Is weakness a signal? True


## No demand - Weakness B
- up bar
- volume: low or high
- spread: low
- close: bottom-third and middle-third

In [9]:
# def is_no_demand_signal(data):
#     if (data['bar_type'] == 'up-bar') and \
#         # ((data['OBV_label']=='high') or (data['OBV_label']=='low')) and \
#         # (data['label_spread'] == 'low') and \
#         ((data['close_bar_label'] == 'bottom-third') or data['close_bar_label'] == 'middle-third'):
#         return True
#     else:
#         return False

In [10]:
def is_no_demand_signal(data):
    if (data['bar_type'] == 'up-bar') and \
        ((data['close_bar_label'] == 'bottom-third') or data['close_bar_label'] == 'middle-third'):
        return True
    else:
        return False

## Up-trust - Pseudo Up-trust
- up bar or down bar 
- high spread
- close: bottom-third
- high or low volume


In [11]:
# def is_up_trust_signal(data):
#     if ((data['OBV_label']=='high') or (data['OBV_label']=='low')) and \
#         (data['label_spread'] == 'high') and \
#         ((data['close_bar_label'] == 'bottom-third') ):
#         return True
#     else:
#         return False

In [12]:
def is_up_trust_signal(data):
    if ((data['OBV_label']=='high') or (data['OBV_label']=='low')) and \
        ((data['close_bar_label'] == 'bottom-third') ):
        return True
    else:
        return False

## Stop volume
- up bar
- high spread
- high volume

In [13]:
# def is_stop_volume_signal(data):
#     if (data['bar_type']=='up-bar') and \
#         (data['OBV_label']=='high') and \
#         (data['label_spread'] == 'high'):
#         return True
#     else:
#         return False

In [14]:
def is_stop_volume_signal(data):
    if (data['bar_type']=='up-bar') and \
        (data['label_spread'] == 'high'):
        return True
    else:
        return False

In [15]:
for i in range(len(data)):
    # Lấy ngày tương ứng
    date = data.loc[i, 'time']
    
    # Áp dụng hàm tương ứng cho ngày tương ứng
    if i % 4 == 0:
        data.loc[i, 'results'] = is_weakness_a_signal(data.loc[i])
        print(f"Step {i}: Date {date} is_weakness_a_signal result = {result}")
    elif i % 4 == 1:
        data.loc[i, 'results'] = is_no_demand_signal(data.loc[i])
        print(f"Step {i}: Date {date} is_no_demand_signal result = {result}")
    elif i % 4 == 2:
        data.loc[i, 'results'] = is_up_trust_signal(data.loc[i])
        print(f"Step {i}: Date {date} is_up_trust_signal result = {result}")
    elif i % 4 == 3:
        data.loc[i, 'results'] = is_stop_volume_signal(data.loc[i])
        print(f"Step {i}: Date {date} is_stop_volume_signal result = {result}")
data.head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_2932\1762669396.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.loc[i, 'results'] = is_weakness_a_signal(data.loc[i])


Step 0: Date 2019-01-02 is_weakness_a_signal result = True
Step 1: Date 2019-01-03 is_no_demand_signal result = True
Step 2: Date 2019-01-04 is_up_trust_signal result = True
Step 3: Date 2019-01-07 is_stop_volume_signal result = True
Step 4: Date 2019-01-08 is_weakness_a_signal result = True
Step 5: Date 2019-01-09 is_no_demand_signal result = True
Step 6: Date 2019-01-10 is_up_trust_signal result = True
Step 7: Date 2019-01-11 is_stop_volume_signal result = True
Step 8: Date 2019-01-14 is_weakness_a_signal result = True
Step 9: Date 2019-01-15 is_no_demand_signal result = True
Step 10: Date 2019-01-16 is_up_trust_signal result = True
Step 11: Date 2019-01-17 is_stop_volume_signal result = True
Step 12: Date 2019-01-18 is_weakness_a_signal result = True
Step 13: Date 2019-01-21 is_no_demand_signal result = True
Step 14: Date 2019-01-22 is_up_trust_signal result = True
Step 15: Date 2019-01-23 is_stop_volume_signal result = True
Step 16: Date 2019-01-24 is_weakness_a_signal result = Tru

,time,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread,close_bar_label,results
0,2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium,bottom-third,False
1,2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low,middle-third,False
2,2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium,top-third,False
3,2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high,bottom-third,True
4,2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high,top-third,False


In [17]:
abc = data[data['results'] == True]
abc.head()

,time,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread,close_bar_label,results
3,2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high,bottom-third,True
7,2019-01-11,35590,35780,35400,35590,1115410,VCB,NaN,NaN,35460.0,up-bar,0.000000,high,middle-third,True
15,2019-01-23,35400,35780,35400,35720,514720,VCB,NaN,NaN,35590.0,up-bar,0.842105,high,top-third,True
16,2019-01-24,35720,35850,35460,35720,463870,VCB,NaN,NaN,35720.0,down-bar,0.000000,high,middle-third,True
17,2019-01-25,35780,36170,35720,35980,1008850,VCB,NaN,NaN,35720.0,up-bar,0.444444,medium,middle-third,True


In [ ]:
data['result'] = False
data['result'] = data.apply(lambda row: is_weakness_a_signal(row), axis=1)
data['result2'] = data.apply(lambda row: is_no_demand_signal(row), axis=1)
data['result3'] = data.apply(lambda row: is_up_trust_signal(row), axis=1)
data['result4'] = data.apply(lambda row: is_stop_volume_signal(row), axis=1)
data.head()

In [ ]:
num_true_rows = (data['result4'] == True).sum()
print("Số dòng có result bằng True:", num_true_rows)

In [18]:
data['results'].value_counts()

results
False    1067
True      207
Name: count, dtype: int64

In [25]:

# Tạo cột "Lệnh"
data['execute_a_trade'] = 'hold'  # Đặt mặc định là "Hold"

window_size = 3  # Kích thước cửa sổ

data['rolling_sum'] = data['results'].rolling(window_size).sum()

data.loc[data['rolling_sum'] == window_size, 'execute_a_trade'] = 'buy'

data = data.drop(columns='rolling_sum')

data.head(10)

,time,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread,close_bar_label,results,execute_a_trade
0,2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium,bottom-third,False,hold
1,2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low,middle-third,False,hold
2,2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium,top-third,False,hold
3,2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high,bottom-third,True,hold
4,2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high,top-third,False,hold
5,2019-01-09,35780,36170,35330,35980,1136370,VCB,NaN,NaN,35460.0,up-bar,0.238095,low,top-third,False,hold
6,2019-01-10,35980,35980,35460,35460,794870,VCB,NaN,NaN,35980.0,down-bar,1.000000,high,bottom-third,False,hold
7,2019-01-11,35590,35780,35400,35590,1115410,VCB,NaN,NaN,35460.0,up-bar,0.000000,high,middle-third,True,hold
8,2019-01-14,35590,35780,35400,35660,759980,VCB,NaN,NaN,35590.0,up-bar,0.184211,low,top-third,False,hold
9,2019-01-15,35660,36170,35590,36040,1071080,VCB,NaN,NaN,35660.0,up-bar,0.655172,medium,top-third,False,hold


In [ ]:
# import pandas as pd

# # Tạo DataFrame mẫu
# data1 = {'ngay': ['2024-02-25', '2024-02-26', '2024-02-27', '2024-02-28', '2024-03-01', '2024-03-02','2024-03-03', '2024-03-04', '2024-03-05'],
#         'results': [False, True, True, True, True, True, False, True, True]}
# df = pd.DataFrame(data1)
# # Tạo cột "Lệnh"
# df['execute_a_trade'] = 'hold'  # Đặt mặc định là "Hold"

# window_size = 4  # Kích thước cửa sổ

# df['rolling_sum'] = df['results'].rolling(window_size).sum()

# df.loc[df['rolling_sum'] == window_size, 'execute_a_trade'] = 'buy'

# df = df.drop(columns='rolling_sum')

# print(df)

In [26]:
data[['time', 'OBV_label', 'bar_type', 'label_spread', 'close_bar_label', 'results', 'execute_a_trade']].head(20)

,time,OBV_label,bar_type,label_spread,close_bar_label,results,execute_a_trade
0,2019-01-02,NaN,down-bar,medium,bottom-third,False,hold
1,2019-01-03,NaN,down-bar,low,middle-third,False,hold
2,2019-01-04,NaN,up-bar,medium,top-third,False,hold
3,2019-01-07,NaN,up-bar,high,bottom-third,True,hold
4,2019-01-08,NaN,up-bar,high,top-third,False,hold
5,2019-01-09,NaN,up-bar,low,top-third,False,hold
6,2019-01-10,NaN,down-bar,high,bottom-third,False,hold
7,2019-01-11,NaN,up-bar,high,middle-third,True,hold
8,2019-01-14,NaN,up-bar,low,top-third,False,hold
9,2019-01-15,NaN,up-bar,medium,top-third,False,hold


In [27]:
data['execute_a_trade'].value_counts()

execute_a_trade
hold    1267
buy        7
Name: count, dtype: int64

# Power pattern

## Power A
- up bar
- medium spread
- medium or high volume
- close: top-third

In [ ]:
def is_power_A_signal(data):
    if (data['bar_type'] == 'up-bar') and \
        (data['label_spread'] == 'medium') and \
        ((data['OBV_label']=='medium') or (data['OBV_label'] == 'low')) and \
        (data['close_bar_label'] == 'top-third'):
        return True
    else:
        return False

## Power B - Lack of order
- down bar 
- low spread 
- low or high volume
- close: top-third or bottom-third

In [ ]:
def is_power_B_signal(data):
    if (data['bar_type'] == 'down-bar') and \
        (data['label_spread'] == 'low') and \
        ((data['OBV_label']=='low') or (data['OBV_label'] == 'high')) and \
        ((data['close_bar_label'] == 'top-third') or (data['close_bar_label'] == 'bottom-third')):
        return True
    else:
        return False

## Reverse Up-Trust - Pseudo Up-trust
- up bar or down bar 
- high spread
- close: bottom-third
- high and low volume


In [ ]:
def is_reverse_up_trust_signal(data):
    if ((data['bar_type'] == 'down-bar') or (data['bar_type']) == 'up-bar') and \
        (data['label_spread'] == 'high') and \
        ((data['OBV_label']=='low') or (data['OBV_label'] == 'high')) and \
        (data['close_bar_label'] == 'bottom-third'):
        return True
    else:
        return False

## Stopped volume
- down bar
- low spread
- close: middle-third
- high volume


In [ ]:
def is_stopped_volume_signal(data):
    if (data['bar_type'] == 'down-bar') and \
        (data['label_spread'] == 'low') and \
        (data['OBV_label'] == 'high') and \
        (data['close_bar_label'] == 'middle-third'):
        return True
    else:
        return False